In [40]:
import re

In [41]:
import pandas as pd

# Load the datasets
rainfall_df = pd.read_csv("data/rainfall_data.csv")
accident_casualties_df = pd.read_csv("data/RoadTrafficAccidentCasualtiesMonthly.csv")
traffic_flow_df = pd.read_csv("data/traffic_flow_data.csv")
accident_locations_df = pd.read_csv("data/Googlemaps_accident_locations.csv")

In [42]:
sample_data = {
    "Rainfall Data": rainfall_df.head(),
    "Accident Casualties": accident_casualties_df.head(),
    "Traffic Flow": traffic_flow_df.head(),
    "Accident Locations": accident_locations_df.head()
}

sample_data

{'Rainfall Data':   station_id         station_name  latitude  longitude  rainfall_mm  \
 0        S77       Alexandra Road    1.2937   103.8125          0.0   
 1       S109  Ang Mo Kio Avenue 5    1.3764   103.8492          0.0   
 2       S117          Banyan Road    1.2560   103.6790          0.0   
 3        S64   Bukit Panjang Road    1.3824   103.7603          0.0   
 4        S90     Bukit Timah Road    1.3191   103.8191          0.0   
 
                    timestamp  
 0  2024-01-01T00:05:00+08:00  
 1  2024-01-01T00:05:00+08:00  
 2  2024-01-01T00:05:00+08:00  
 3  2024-01-01T00:05:00+08:00  
 4  2024-01-01T00:05:00+08:00  ,
 'Accident Casualties':                             DataSeries  2025Feb  2025Jan  2024Dec  2024Nov  \
 0          Total Casualties Fatalities        9       12       16       15   
 1                          Pedestrians        1        4        2        2   
 2       Personal Mobility Device Users        0        0        0        0   
 3               

In [43]:


# === Step 2: Extract date from the 'Location' field ===
def extract_date(location_str):
    match = re.search(r'(\d{4}-\d{2}-\d{2})', str(location_str))
    if match:
        return pd.to_datetime(match.group(1))
    return None

# Apply the extraction
accident_locations_df['Date'] = accident_locations_df['Location'].apply(extract_date)

# === Step 3: Drop rows with invalid/missing dates ===
accident_locations_df = accident_locations_df.dropna(subset=['Date'])

# === Step 4: Select required columns ===
node_features_df = accident_locations_df[['Latitude', 'Longitude', 'Date']].copy()
node_features_df.reset_index(drop=True, inplace=True)

print(node_features_df.head())



   Latitude   Longitude       Date
0  1.297283  103.848905 2024-06-15
1  1.320541  103.880998 2024-06-14
2  1.355467  103.720976 2024-06-13
3  1.349250  103.837236 2024-06-12
4  1.321988  103.670277 2024-06-11


RAINFALL

In [44]:
import numpy as np
from geopy.distance import geodesic
import pandas as pd

# Assuming node_features_df is already in memory
# Also assuming rainfall_df is already loaded:
# rainfall_df = pd.read_csv("rainfall_data.csv")

# Ensure timestamp is in datetime format
rainfall_df['timestamp'] = pd.to_datetime(rainfall_df['timestamp'])

# Function to get rainfall at nearest station and same day
def get_rainfall(lat, lon, date):
    # Filter rainfall on the same day
    same_day = rainfall_df[rainfall_df['timestamp'].dt.date == date.date()]
    if same_day.empty:
        return np.nan

    # Compute distances to all stations
    same_day['distance'] = same_day.apply(
        lambda row: geodesic((lat, lon), (row['latitude'], row['longitude'])).meters,
        axis=1
    )

    # Select closest station's value
    closest = same_day.sort_values(by=['distance', 'timestamp']).iloc[0]
    return closest['rainfall_mm']

# Apply to each accident node
node_features_df['Rainfall_mm'] = node_features_df.apply(
    lambda row: get_rainfall(row['Latitude'], row['Longitude'], row['Date']),
    axis=1
)

# Preview updated DataFrame
print(node_features_df.head())


   Latitude   Longitude       Date  Rainfall_mm
0  1.297283  103.848905 2024-06-15          0.0
1  1.320541  103.880998 2024-06-14          0.0
2  1.355467  103.720976 2024-06-13          0.0
3  1.349250  103.837236 2024-06-12          0.0
4  1.321988  103.670277 2024-06-11          0.0


TRAFFIC VOLUME

In [45]:
from math import radians, cos, sin, asin, sqrt

# --- Haversine Function ---
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371000 * c  # meters

# --- Preprocess traffic data ---
traffic_df = traffic_flow_df.copy()
traffic_df['Date'] = pd.to_datetime(traffic_df['Date'], format='%d/%m/%Y', errors='coerce')
traffic_df = traffic_df.dropna(subset=['Date'])

# Get list of Sundays in traffic data
available_traffic_dates = traffic_df['Date'].dt.date.unique()

# Function to find nearest Sunday
def find_nearest_traffic_date(accident_date):
    accident_day = accident_date.date()
    nearest = min(available_traffic_dates, key=lambda d: abs((accident_day - d).days))
    return pd.to_datetime(nearest)

# --- Function to get traffic volume ---
def get_traffic_volume(lat, lon, accident_date):
    nearest_date = find_nearest_traffic_date(accident_date)
    same_day = traffic_df[traffic_df['Date'].dt.date == nearest_date.date()]
    
    if same_day.empty:
        return None
    
    # Compute distance from node to road segment start point
    same_day['distance'] = same_day.apply(
        lambda row: haversine(lon, lat, row['StartLon'], row['StartLat']),
        axis=1
    )

    closest = same_day.sort_values('distance').iloc[0]
    return closest['Volume']

# --- Apply to all node rows ---
node_features_df['Traffic_Volume'] = node_features_df.apply(
    lambda row: get_traffic_volume(row['Latitude'], row['Longitude'], row['Date']),
    axis=1
)

# Done! Preview
print(node_features_df.head())


   Latitude   Longitude       Date  Rainfall_mm  Traffic_Volume
0  1.297283  103.848905 2024-06-15          0.0            3942
1  1.320541  103.880998 2024-06-14          0.0            4733
2  1.355467  103.720976 2024-06-13          0.0            3575
3  1.349250  103.837236 2024-06-12          0.0            1556
4  1.321988  103.670277 2024-06-11          0.0            1968


CASUALTY

In [46]:
import pandas as pd

# Step 1: Work on a copy of the casualties DataFrame
casualties_df = accident_casualties_df.copy()

# Step 2: Transpose so months become rows
casualties_df.set_index("DataSeries", inplace=True)
casualties_df = casualties_df.T  # Now each row is a month
casualties_df.index.name = "Month"
casualties_df.reset_index(inplace=True)

# Step 3: Convert "Month" strings like "2024Jun" to datetime
def parse_month_string(s):
    try:
        return pd.to_datetime(s, format="%Y%b")
    except:
        return None

casualties_df['Date'] = casualties_df['Month'].apply(parse_month_string)
casualties_df.dropna(subset=['Date'], inplace=True)

# Step 4: Align to monthly resolution
casualties_df['Month'] = casualties_df['Date'].dt.to_period('M').dt.to_timestamp()
node_features_df['Month'] = node_features_df['Date'].dt.to_period('M').dt.to_timestamp()

# Step 5: Merge casualties into node features
merged_df = pd.merge(node_features_df, casualties_df, on='Month', how='left')

# # Step 6: Rename useful columns (optional)
# # Fix column renaming based on actual keys in merged_df
# merged_df.rename(columns={
#     'Total casualties: Fatalities': 'Fatalities',
#     'Total casualties: Pedestrians': 'Pedestrian_Casualties',
#     'Motor Cyclists & Pillion Riders': 'Motorcyclist_Casualties'
# }, inplace=True)


# Step 7: Clean up
merged_df.drop(columns=['Date_y'], inplace=True)
merged_df.rename(columns={'Date_x': 'Date'}, inplace=True)

# Step 8: Replace node_features_df with enriched version
node_features_df = merged_df

# ✅ Preview result
print(node_features_df.head())


   Latitude   Longitude       Date  Rainfall_mm  Traffic_Volume      Month  \
0  1.297283  103.848905 2024-06-15          0.0            3942 2024-06-01   
1  1.320541  103.880998 2024-06-14          0.0            4733 2024-06-01   
2  1.355467  103.720976 2024-06-13          0.0            3575 2024-06-01   
3  1.349250  103.837236 2024-06-12          0.0            1556 2024-06-01   
4  1.321988  103.670277 2024-06-11          0.0            1968 2024-06-01   

  Total Casualties Fatalities     Pedestrians  \
0                          13               2   
1                          13               2   
2                          13               2   
3                          13               2   
4                          13               2   

      Personal Mobility Device Users     Cyclists & Pillions  ...  \
0                                  0                       1  ...   
1                                  0                       1  ...   
2                            

In [47]:
import pandas as pd

# Step 1: Transpose casualties so each month is a row
casualties_df = accident_casualties_df.copy()
casualties_df.set_index("DataSeries", inplace=True)
casualties_df = casualties_df.T  # now months are rows
casualties_df.index.name = "Month"
casualties_df.reset_index(inplace=True)

# Step 2: Convert "2024Jun" style to datetime
def parse_month_string(s):
    try:
        return pd.to_datetime(s, format="%Y%b")
    except:
        return None

casualties_df['Date'] = casualties_df['Month'].apply(parse_month_string)
casualties_df.dropna(subset=['Date'], inplace=True)

# Step 3: Create 'Month' key in both datasets for merging
casualties_df['Month'] = casualties_df['Date'].dt.to_period('M').dt.to_timestamp()
node_features_df['Month'] = node_features_df['Date'].dt.to_period('M').dt.to_timestamp()

# Step 4: Merge by Month
merged_df = pd.merge(node_features_df, casualties_df, on='Month', how='left', suffixes=('', '_casualty'))

# Step 5: Clean up and restore 'Date' column name
# merged_df.drop(columns=['Date_y'], inplace=True)
# merged_df.rename(columns={'Date_x': 'Date'}, inplace=True)

# Step 6: Assign merged result back to node_features_df
node_features_df = merged_df



print(node_features_df.head())


   Latitude   Longitude       Date  Rainfall_mm  Traffic_Volume      Month  \
0  1.297283  103.848905 2024-06-15          0.0            3942 2024-06-01   
1  1.320541  103.880998 2024-06-14          0.0            4733 2024-06-01   
2  1.355467  103.720976 2024-06-13          0.0            3575 2024-06-01   
3  1.349250  103.837236 2024-06-12          0.0            1556 2024-06-01   
4  1.321988  103.670277 2024-06-11          0.0            1968 2024-06-01   

  Total Casualties Fatalities     Pedestrians  \
0                          13               2   
1                          13               2   
2                          13               2   
3                          13               2   
4                          13               2   

      Personal Mobility Device Users     Cyclists & Pillions  ...  \
0                                  0                       1  ...   
1                                  0                       1  ...   
2                            

In [48]:
node_features_df.drop(columns=['Month'], inplace=True, errors='ignore')
node_features_df.to_csv("data/node_features.csv", index=False)